# Dataset Download

In [1]:
# load the iris dataset from pycaret repo
from pycaret.datasets import get_data
data = get_data('iris')

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# 1. PyCaret

### 1-1. Initialize Setup

In [2]:
# initialize setup
from pycaret.classification import *
s = setup(data, target = 'species', session_id = 123)

,Description,Value
0,session_id,123
1,Target,species
2,Target Type,Multiclass
3,Label Encoded,"Iris-setosa: 0, Iris-versicolor: 1, Iris-virginica: 2"
4,Original Data,"(150, 5)"
5,Missing Values,False
6,Numeric Features,4
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


### 1-2. Compare ML models

In [3]:
best = compare_models(n_select = 15)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9809,0.9969,0.9833,0.9857,0.9809,0.9715,0.9739,0.0040
knn,K Neighbors Classifier,0.9800,0.9830,0.9806,0.9800,0.9800,0.9697,0.9697,0.2800
qda,Quadratic Discriminant Analysis,0.9709,0.9969,0.9750,0.9782,0.9709,0.9566,0.9602,0.0040
lr,Logistic Regression,0.9609,0.9921,0.9611,0.9622,0.9596,0.9403,0.9422,0.7560
nb,Naive Bayes,0.9609,0.9938,0.9611,0.9652,0.9605,0.9407,0.9432,0.0030
gbc,Gradient Boosting Classifier,0.9609,0.9782,0.9611,0.9723,0.9579,0.9402,0.9476,0.0370
dt,Decision Tree Classifier,0.9509,0.9616,0.9500,0.9598,0.9479,0.9249,0.9309,0.0030
et,Extra Trees Classifier,0.9509,0.9890,0.9528,0.9532,0.9509,0.9258,0.9269,0.0380
rf,Random Forest Classifier,0.9409,0.9875,0.9417,0.9422,0.9396,0.9100,0.9119,0.0420
ada,Ada Boost Classifier,0.9409,0.9895,0.9417,0.9467,0.9391,0.9100,0.9146,0.0170


In [4]:
# pull() stores the output of compare_model() as pd.DataFrame
compare_model_results = pull()

In [5]:
compare_model_results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lda,Linear Discriminant Analysis,0.9809,0.9969,0.9833,0.9857,0.9809,0.9715,0.9739,0.004
knn,K Neighbors Classifier,0.9800,0.9830,0.9806,0.9800,0.9800,0.9697,0.9697,0.280
qda,Quadratic Discriminant Analysis,0.9709,0.9969,0.9750,0.9782,0.9709,0.9566,0.9602,0.004
lr,Logistic Regression,0.9609,0.9921,0.9611,0.9622,0.9596,0.9403,0.9422,0.756
nb,Naive Bayes,0.9609,0.9938,0.9611,0.9652,0.9605,0.9407,0.9432,0.003
gbc,Gradient Boosting Classifier,0.9609,0.9782,0.9611,0.9723,0.9579,0.9402,0.9476,0.037
dt,Decision Tree Classifier,0.9509,0.9616,0.9500,0.9598,0.9479,0.9249,0.9309,0.003
et,Extra Trees Classifier,0.9509,0.9890,0.9528,0.9532,0.9509,0.9258,0.9269,0.038
rf,Random Forest Classifier,0.9409,0.9875,0.9417,0.9422,0.9396,0.9100,0.9119,0.042
ada,Ada Boost Classifier,0.9409,0.9895,0.9417,0.9467,0.9391,0.9100,0.9146,0.017


In [6]:
len(best)

15

In [7]:
best[:5]

[LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                            solver='svd', store_covariance=False, tol=0.0001),
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                      weights='uniform'),
 QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                               store_covariance=False, tol=0.0001),
 LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=1000,
                    multi_class='auto', n_jobs=None, penalty='l2',
                    random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                    warm_start=False),
 GaussianNB(priors=None, var_smoothing=1e-09)]

# 2. Gradio

### 2-1. Creating an internal function

In [8]:
# creating a predict function to be passed into gradio UI
def predict(model, sepal_length, sepal_width, petal_length, petal_width):
  
    df = pd.DataFrame.from_dict({'sepal_length': [sepal_length], 'sepal_width': [sepal_width], 
                                 'petal_length': [petal_length], 'petal_width': [petal_width]})
    
    model_index = list(compare_model_results['Model']).index(model)
    model = best[model_index]
    pred = predict_model(model, df, raw_score=True)
    
    # return pred['~~~'][0] because there will be only one prediction(probability) for each category
    return {'Iris-setosa': pred['Score_Iris-setosa'][0].astype('float64'), 
            'Iris-versicolor': pred['Score_Iris-versicolor'][0].astype('float64'),
            'Iris-virginica' : pred['Score_Iris-virginica'][0].astype('float64')}

### 2-2. Create a UI with Gradio

In [9]:
import gradio as gr

model = gr.inputs.Dropdown(
    list(compare_model_results['Model']), 
    label="Model"
)
sepal_length = gr.inputs.Slider(
    minimum=1, 
    maximum=10, 
    default=data['sepal_length'].mean(), 
    label = 'sepal_length'
)
sepal_width = gr.inputs.Slider(
    minimum=1, 
    maximum=10, 
    default=data['sepal_width'].mean(), 
    label = 'sepal_width'
)
petal_length = gr.inputs.Slider(
    minimum=1, 
    maximum=10, 
    default=data['petal_length'].mean(), 
    label = 'petal_length'
)
petal_width = gr.inputs.Slider(
    minimum=1, 
    maximum=10, 
    default=data['petal_width'].mean(), 
    label = 'petal_width'
)

gr.Interface(
    predict, 
    [model, sepal_length, sepal_width, petal_length, petal_width], # These are the arguments for the function predict()
    "label", 
    live=True
).launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x210e5beb040>, 'http://127.0.0.1:7860/', None)